In [239]:
import pandas as pd

In [240]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [241]:
df = df.sort_values("date") #sorts all values by date 

In [242]:
df = df.reset_index(drop=True) #cleans up table

In [243]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"] #deletes extraneous data

In [244]:
def add_target(team):
    team["target"] = team["won"].shift(-1) #pulls the won column from the last game
    return team

df = df.groupby("team", group_keys = False).apply(add_target)

/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/986073505.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1) #pulls the won column from the last game
/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/986073505.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1) #pulls the won column from the last game
/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/986073505.py:2: PerformanceWarning: DataFrame is highly

In [245]:
df["target"][pd.isnull(df["target"])] = 2 #sets the target category to 2 if there is NaN

/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/3950488221.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2 #sets the target category to 2 if there is NaN
/var/folders/1t/3fj79vz1

In [246]:
df["target"] = df["target"].astype(int,errors="ignore")

In [247]:
nulls = pd.isnull(df)

In [248]:
nulls = nulls.sum()

In [249]:
nulls = nulls[nulls>0] 

In [250]:
valid_columns = df.columns[~df.columns.isin(nulls.index)] #checks if dataframe column is in the nulls index

In [251]:
df = df[valid_columns].copy()

In [252]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select =30, direction = "forward", cv = split)

In [253]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [254]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [255]:
sfs.fit(df[selected_columns],df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [256]:
predictors = list(selected_columns[sfs.get_support()])

In [257]:
def backtest(data,model,predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"]) 
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [258]:
predictions = backtest(df, rr, predictors)

In [259]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

0.5471969333972209

In [260]:
df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/330557953.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


home
0.0    0.428314
1.0    0.571686
dtype: float64

In [261]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,0.277279,0.554502,0.317647,GSW,0.451923,1.0,2016,2015-10-27,False,0
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,0.160462,0.345972,0.317647,CHI,0.317308,1.0,2016,2015-10-27,False,1
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,0.088575,0.232227,0.329412,CLE,0.298077,0.0,2016,2015-10-27,True,1
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,0.215661,0.530806,0.505882,NOP,0.298077,0.0,2016,2015-10-27,True,1
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,0.019255,0.203791,0.317647,DET,0.403846,0.0,2016,2015-10-27,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.182285,0.208531,0.411765,GSW,0.413462,0.0,2022,2022-06-10,False,0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.928113,1.000000,0.411765,BOS,0.288462,0.0,2022,2022-06-13,True,1
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.181001,0.630332,0.352941,GSW,0.384615,1.0,2022,2022-06-13,False,0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.120668,0.459716,0.400000,GSW,0.375000,0.0,2022,2022-06-16,False,2


In [262]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [263]:
def find_team_averages(team):
    # Select only numeric columns
    numeric_columns = team.select_dtypes(include='number')
    # Compute the rolling mean on the numeric columns
    rolling = numeric_columns.rolling(10).mean()
    # If you want to keep the non-numeric columns, you can join them back after rolling mean calculation
    non_numeric_columns = team.select_dtypes(exclude='number').iloc[-1:]
    result = rolling.join(non_numeric_columns)
    return result

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/4068766723.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [264]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [265]:
df = df.dropna()

In [266]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/1t/3fj79vz15k12cjnzspxhr_380000gn/T/ipykernel_87352/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(

In [267]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,NYK,PHI,PHI,NYK,2017-04-12
1,ORL,DET,DET,ORL,2017-04-12
2,PHI,NYK,NYK,PHI,2017-04-12
3,DET,ORL,ORL,DET,2017-04-12
4,CLE,GSW,GSW,CLE,2017-06-12
5,GSW,CLE,CLE,GSW,2017-06-12
6,PHO,DAL,DAL,PHO,2018-04-10
7,LAL,LAC,LAC,LAL,2018-04-11
8,DET,CHI,CHI,DET,2018-04-11
9,DAL,PHO,PHO,DAL,2018-04-10


In [268]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [269]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [270]:
predictors = list(selected_columns[sfs.get_support()])

In [271]:
predictors

['mp',
 'fg',
 'fga',
 'fg%',
 '3p',
 '3pa',
 '3p%',
 'ft',
 'fta',
 'ft%',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'ts%',
 'efg%',
 '3par',
 'ftr',
 'orb%',
 'drb%',
 'tov%',
 'usg%',
 'drtg',
 '3p_max',
 'ortg_max']

In [272]:
predictions = backtest(full, rr, predictors)

In [273]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.5833333333333334